In [10]:
import sys
import torch
import os
import pprint
from tqdm import tqdm
pprint.pprint(sys.path)


['C:\\Users\\hirom\\NLP\\LLM\\fcc-gpt-course',
 'C:\\Users\\hirom\\anaconda3\\python39.zip',
 'C:\\Users\\hirom\\anaconda3\\DLLs',
 'C:\\Users\\hirom\\anaconda3\\lib',
 'C:\\Users\\hirom\\anaconda3',
 '',
 'C:\\Users\\hirom\\AppData\\Roaming\\Python\\Python39\\site-packages',
 'C:\\Users\\hirom\\anaconda3\\lib\\site-packages',
 'C:\\Users\\hirom\\anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\hirom\\anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\hirom\\anaconda3\\lib\\site-packages\\Pythonwin',
 'D:\\NLP\\JParaCrawl',
 'C:\\users\\hirom\\nlp\\llm\\cuda\\lib\\site-packages']


In [3]:
sys.path.append('D:\\NLP\\JParaCrawl')
sys.path.append('C:\\users\\hirom\\nlp\\llm\cuda\\lib\\site-packages')

In [4]:
pprint.pprint(sys.path)

['C:\\Users\\hirom\\NLP\\LLM\\fcc-gpt-course',
 'C:\\Users\\hirom\\anaconda3\\python39.zip',
 'C:\\Users\\hirom\\anaconda3\\DLLs',
 'C:\\Users\\hirom\\anaconda3\\lib',
 'C:\\Users\\hirom\\anaconda3',
 '',
 'C:\\Users\\hirom\\AppData\\Roaming\\Python\\Python39\\site-packages',
 'C:\\Users\\hirom\\anaconda3\\lib\\site-packages',
 'C:\\Users\\hirom\\anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\hirom\\anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\hirom\\anaconda3\\lib\\site-packages\\Pythonwin',
 'D:\\NLP\\JParaCrawl',
 'C:\\users\\hirom\\nlp\\llm\\cuda\\lib\\site-packages']


In [5]:
from transformers import BertTokenizer, BertConfig, BertForMaskedLM, AdamW, get_linear_schedule_with_warmup

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [7]:
def load_data(file_path):
    lines = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            lines.append(line.strip())
    return lines

en_train_path = 'D:\\NLP\\JParaCrawl\\JP_en_train.txt'
en_test_path = 'D:\\NLP\\JParaCrawl\\JP_en_test.txt'
ja_train_path = 'D:\\NLP\\JParaCrawl\\JP_ja_train.txt'
ja_test_path = 'D:\\NLP\\JParaCrawl\\JP_ja_test.txt'

en_train = load_data(en_train_path)
en_test = load_data(en_test_path)
ja_train = load_data(ja_train_path)
ja_test = load_data(ja_test_path)

In [8]:
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
config = BertConfig.from_pretrained(model_name)
bert_masked_model = BertForMaskedLM.from_pretrained(model_name, config=config).to(device)

C:\users\hirom\nlp\llm\cuda\lib\site-packages\huggingface_hub\file_download.py:982: UserWarning: Not enough free disk space to download the file. The expected file size is: 714.29 MB. The target location C:\Users\hirom\.cache\huggingface\hub\models--bert-base-multilingual-cased\blobs only has 547.57 MB free disk space.
  warnings.warn(


model.safetensors:  70%|#######   | 503M/714M [00:00<?, ?B/s]

C:\users\hirom\nlp\llm\cuda\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hirom\.cache\huggingface\hub\models--bert-base-multilingual-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForM

In [11]:
def get_max_length(text_list, tokenizer):
    max_length = 0
    for text in tqdm(text_list, desc="processing texts"):
        encoded_text = tokenizer(text, add_special_tokens=True, padding=False, truncation=False)
        text_length = len(encoded_text['input_ids'])
        max_length = max(max_length, text_length)
    return max_length

en_train_max_length = get_max_length(en_train, tokenizer)
en_test_max_length = get_max_length(en_test, tokenizer)
ja_train_max_length = get_max_length(ja_train, tokenizer)
ja_test_max_length = get_max_length(ja_test, tokenizer)

max_length = max(en_train_max_length, en_test_max_length, ja_train_max_length, ja_test_max_length)
print(f"Optimal max_length: {max_length}")

processing texts: 100%|██████████████████████████████████████████████████████| 876400/876400 [08:53<00:00, 1641.49it/s]

Optimal max_length: 772
